In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
import pandas as pd
import matplotlib as plt
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import requests
from time import sleep  # Optional: be polite to the API

data = pd.read_csv('top10K-TMDB-movies.csv')
df = pd.DataFrame(data)
# find and remove mssing data

new_data = df.dropna()

new_data['final_tags'] = new_data['genre'] + new_data['overview']

final_data = new_data.drop(['genre', 'original_language', 'overview', 'popularity',
       'vote_average', 'vote_count','release_date',],axis=1)

tfvectorize = TfidfVectorizer(stop_words='english')
tf_matrix = tfvectorize.fit_transform(final_data['final_tags'])

cosine_func = cosine_similarity(tf_matrix)


print(cosine_func[0])


from sklearn.neighbors import NearestNeighbors


model = NearestNeighbors(n_neighbors=11, metric='cosine')
model.fit(tf_matrix)

indices = pd.Series(final_data.index, final_data['title'])


def reccommend_movie_KN(title,n):
    idx = indices[title]
    # print(idx)

    incidies_list = model.kneighbors(tf_matrix[idx], n_neighbors=11)
    # print(incidies_list[1])
    result = []

    for movie in incidies_list[1]:
        # print(incidies_list[1])
        movie_list = list( final_data['title'].iloc[movie])
        movie = movie_list[1:n+1]
        return movie


import requests
from time import sleep  # Optional: be polite to the API

API_KEY = 'a01b8453d45c3f5806c7397aa4addae5'
BASE_URL = 'https://api.themoviedb.org/3/search/movie'

# recommended = ['Interstellar', 'Inception', 'The Matrix', 'Titanic', 'Avatar',
#                'The Dark Knight', 'Fight Club', 'Forrest Gump', 'Gladiator', 'Up']

movie_list = reccommend_movie_KN("Spider-Man",5)

def get_movies(title):
    params = {
        'api_key': API_KEY,
        'query': title
    }

    res = requests.get(BASE_URL, params= params)
    data = res.json()
    # print(data['results'])

    if data['results']:
        movie = data['results'][0]
        return {
            'title': movie['title'],
            'poster_url': f"https://image.tmdb.org/t/p/w500{movie['poster_path']}" if movie.get('poster_path') else None,
            'rating': movie['vote_average'],
            'overview': movie['overview'],
            'release_date': movie['release_date']
        }
    return {'title': title, 'error': 'Not found'}

# get_movies(title='')
total_movies_list =[]

for m in movie_list:
    movie = get_movies(m)
    total_movies_list.append(movie)
    sleep(0.2)

for movie in total_movies_list:
    print(movie)





C:\Users\sakthi\AppData\Local\Temp\ipykernel_13300\508950430.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data['final_tags'] = new_data['genre'] + new_data['overview']


[1.         0.01078192 0.00778483 ... 0.04233725 0.02887264 0.01411831]
{'title': 'Spider-Man 3', 'poster_url': 'https://image.tmdb.org/t/p/w500/qFmwhVUoUSXjkKRmca5yGDEXBIj.jpg', 'rating': 6.439, 'release_date': '2007-05-01'}
{'title': 'Arachnophobia', 'poster_url': 'https://image.tmdb.org/t/p/w500/lHjpwv1Yv2RcGcTQb4Lc1cKw1y9.jpg', 'rating': 6.433, 'release_date': '1990-07-20'}
{'title': 'Spider-Man: Homecoming', 'poster_url': 'https://image.tmdb.org/t/p/w500/c24sv2weTHPsmDa7jEMN0m2P3RT.jpg', 'rating': 7.331, 'release_date': '2017-07-05'}
{'title': 'Spider-Man: Into the Spider-Verse', 'poster_url': 'https://image.tmdb.org/t/p/w500/iiZZdoQBEYBv6id8su7ImL0oCbD.jpg', 'rating': 8.398, 'release_date': '2018-12-06'}
{'title': 'The Amazing Spider-Man', 'poster_url': 'https://image.tmdb.org/t/p/w500/jexoNYnPd6vVrmygwF6QZmWPFdu.jpg', 'rating': 6.716, 'release_date': '2012-06-23'}
